In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

## Load the excel file into a dataframe 

In [28]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df["Revenue"] = df["Total Environmental Cost"]/df["Environmental Intensity (Sales)"]
df["Operating Income"] = df["Total Environmental Cost"]/df["Environmental Intensity (Op Inc)"]
df["Environmental Intensity (Op Inc)"] = df["Environmental Intensity (Op Inc)"]*100
df["Environmental Intensity (Sales)"] = df["Environmental Intensity (Sales)"]*100
df.dropna()
df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,244464000.0
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,56797000.0
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,638704000.0
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.462497,NaN,-2.290124e+06,-2.239814e+06,-510.210093,...,-0.189720,-1.009642,-253.366805,-0.226133,-3.169102,-3.169102,-92.619013,0.016390,1.565900e+08,NaN
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,784560000.0


## Filter out the outliers in the data

In [54]:
Q1 = df['Environmental Intensity (Op Inc)'].quantile(0.25)
Q3 = df['Environmental Intensity (Op Inc)'].quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers based on the IQR method
df_cleaned = df[(df['Environmental Intensity (Op Inc)'] >= lower_bound) & (df['Environmental Intensity (Op Inc)'] <= upper_bound)]


## Function to run the Fixed Effects and return the r squared value

In [52]:
def runFixedEffects(X, y, df_selected, list_effectNames):
    list_df_Effects = list()

    # Create as list of the dummy variables for the fixed effects 
    for columnName in list_effectNames:
        df_temp = pd.get_dummies(df_selected[columnName], drop_first=True)
        df_temp = df_temp.astype(int)
        list_df_Effects.append(df_temp)
    
    # Add the dummies to the independent variables
    X = pd.concat(list_df_Effects, axis=1)
    # Add a constant (intercept) to the independent variables
    X = sm.add_constant(X)
    
    # Fit the OLS model
    model = sm.OLS(y, X).fit() 
    
    # Calculate  and return the adjusted R-squared
    adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)
    
    return adjusted_r_squared

## setup which dependent and independent variables to use

In [55]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

independentVariables = sdg_set

In [65]:
y_sales = df_cleaned["Environmental Intensity (Sales)"]
y_opinc = df_cleaned["Environmental Intensity (Op Inc)"]
X = df_cleaned[independentVariables]

## Fixed Effect: Year
#### sales

In [66]:
list_effectNames = ['Year']
adjusted_r_squared = runFixedEffects(X,y_sales,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year): {adjusted_r_squared:.6f}')

Adjusted R-squared (Fixed on Year): 0.000044


#### op income

In [67]:
list_effectNames = ['Year']
adjusted_r_squared = runFixedEffects(X,y_opinc,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year): {adjusted_r_squared:.6f}')

Adjusted R-squared (Fixed on Year): -0.000587


## Fixed Effect: Year and GICS Sub-Industry
#### sales

In [68]:
list_effectNames = ['Year', 'GICS Sub-Industry']
adjusted_r_squared = runFixedEffects(X,y_sales,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and GICS Sub-Industry): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and GICS Sub-Industry): 0.3995


#### op income

In [69]:
list_effectNames = ['Year', 'GICS Sub-Industry']
adjusted_r_squared = runFixedEffects(X,y_opinc,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and GICS Sub-Industry): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and GICS Sub-Industry): 0.4150


## Fixed Effect: Year and Industry (Exiobase)
#### sales

In [70]:
list_effectNames = ['Year', 'Industry (Exiobase)']
adjusted_r_squared = runFixedEffects(X,y_sales,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and Industry (Exiobase)): 0.3108


#### op income

In [71]:
list_effectNames = ['Year', 'Industry (Exiobase)']
adjusted_r_squared = runFixedEffects(X,y_opinc,df_cleaned, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and Industry (Exiobase)): 0.3334


## Same effects but only the brewers

In [39]:
df_GICSBrew = df_cleaned[df_cleaned['GICS Sub-Industry'] == 'Brewers']

# Group the data by "Company Name" and count unique years for each group
company_years_count = df_GICSBrew.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
df_GICSBrewFiltered = df_GICSBrew[df_GICSBrew["Company Name"].isin(filtered_companies)]

df_GICSBrewFiltered.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,-6.978419e+08,-2.950986e+08,-77656.206578,...,-1113.208360,-326.988191,-37024.568588,-1326.865949,15176.840942,15176.840942,-12261.768244,0.059202,1.057940e+10,1.500000e+09
523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,-1.778566e+09,-1.561186e+09,-391750.191978,...,-3914.291085,-325.789140,-190769.424529,-4665.559245,56659.806086,56659.806086,-64862.799133,0.080199,5.248464e+10,1.547524e+10
526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,-2.388141e+08,-1.962082e+08,-60262.712495,...,-1480.449400,-288.844325,-27951.868900,-1764.591400,19253.589023,19132.282417,-8148.334758,0.029599,9.890572e+09,1.487489e+09
530,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.898778,-12.419412,-5.102237e+08,-4.329371e+08,-113003.339951,...,-1458.522182,-168.769751,-54528.158099,-1738.455701,20656.408130,20656.408130,-18050.980779,0.093336,2.687117e+10,4.108276e+09
532,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.709847,-10.508374,-4.594560e+08,-4.332461e+08,-113074.003951,...,-1458.522182,-168.769751,-54563.490099,-1738.455701,20656.408130,20656.408130,-18063.841679,0.101854,2.687117e+10,4.372284e+09


In [60]:
y_brew_sales = df_cleaned["Environmental Intensity (Sales)"]
y_brew_opinc = df_cleaned["Environmental Intensity (Op Inc)"]
X_brew = df_GICSBrewFiltered[independentVariables]

In [61]:
list_effectNames = ['Year']
adjusted_r_squared = runFixedEffects(X_brew,y_brew_sales,df_GICSBrewFiltered, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year): {adjusted_r_squared:.6f}')

Adjusted R-squared (Fixed on Year): -0.152248


In [62]:
list_effectNames = ['Year', 'GICS Sub-Industry']
adjusted_r_squared = runFixedEffects(X_brew,y_brew_sales,df_GICSBrewFiltered, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and GICS Sub-Industry): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and GICS Sub-Industry): -0.1522


In [63]:
list_effectNames = ['Year', 'Industry (Exiobase)']
adjusted_r_squared = runFixedEffects(X_brew,y_brew_sales,df_GICSBrewFiltered, list_effectNames)
print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year and Industry (Exiobase)): -0.1522
